In [1]:
!pip install -U liwc

Requirement already up-to-date: liwc in /Users/robertlewis/anaconda3/lib/python3.6/site-packages (0.4.1)
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import nltk
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [3]:
EntropyData = pd.read_excel('4-12-2018_Combined.xlsx')
EntropyData.head()

,Data,emotionally,treated,lovecountry,respect,purity,math,weak,unfairly,betray,...,Round2,Round3,Round4,Gender,Age,Language,Marital_Status,Country,Distracted,Device
0,0,6,6,4,2,5,1,6,5,4,...,6,6,7,2.0,43.0,1.0,3.0,184.0,4.0,4.0
1,0,5,2,1,4,6,1,5,6,4,...,2,2,2,2.0,36.0,1.0,6.0,116.0,2.0,2.0
2,0,4,5,1,2,1,2,4,5,2,...,2,3,7,2.0,28.0,1.0,3.0,185.0,1.0,2.0
3,0,6,6,1,4,5,1,6,6,5,...,4,5,5,1.0,21.0,10.0,6.0,48.0,1.0,3.0
4,0,4,4,1,2,4,1,5,5,4,...,4,4,4,1.0,29.0,10.0,6.0,140.0,1.0,2.0


In [4]:
EntropyData.columns.values

array(['Data', 'emotionally', 'treated', 'lovecountry', 'respect',
       'purity', 'math', 'weak', 'unfairly', 'betray', 'traditions',
       'disgusting', 'cruel', 'rights', 'loyalty', 'chaos', 'god',
       'compassion', 'governmentfairly', 'history', 'kidrespect',
       'harmlessdg', 'good', 'killanimal', 'justice', 'family', 'gender',
       'unnatural', 'kill', 'rich_kids', 'team', 'soldier', 'chastity',
       'Conservatism_1', 'CONDITION_H', 'nonintimate_Backgroundperceive',
       'NonintBgpwordcount', 'nonintimate_BackgroundperceiveFirstclick',
       'nonintimate_BackgroundperceiveLastclick',
       'nonintimate_BackgroundperceivePagesubmite',
       'nonintimate_BackgroundperceiveTotalclicks',
       'Nonintimate_Internetforget', 'NonintForgetwordcount',
       'Nonintimate_InternetforgetFirstclick',
       'Nonintimate_InternetforgetLastclick',
       'Nonintimate_InternetforgetPagesubmit',
       'Nonintimate_InternetforgetTotalclicks',
       'Nonintimatecreativityimpor

In [5]:
# EntropyData['nonintimate_Backgroundperceive']
# EntropyData['Nonintimate_Internetforget']
# EntropyData['Nonintimatecreativityimportant']
# EntropyData['Intimate_embarrassed']
# EntropyData['Intimate_forget']
# EntropyData['Intimate_personalcause']

def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [6]:

import re
from collections import Counter
import liwc

In [7]:
parse, category_names = liwc.load_token_parser('liwc/liwc-english-mod.dic')

In [8]:
# EntropyData['Intimate_embarrassed'][29:445]
print(EntropyData['nonintimate_Backgroundperceive'].isna().sum())
print(EntropyData['Nonintimate_Internetforget'].isna().sum())
print(EntropyData['Nonintimatecreativityimportant'].isna().sum())
print(EntropyData['Intimate_embarrassed'].isna().sum())
print(EntropyData['Intimate_forget'].isna().sum())
print(EntropyData['Intimate_personalcause'].isna().sum())

0
0
0
1
0
0


In [9]:
# replace the NaNs in Intimate_embarassed (with something not in LIWC dic
# so ...
# it doesn't gum up the works below
from numpy import *

Lines = []
for line in EntropyData['Intimate_embarrassed']:
    if line != line: #if it isn't equal to itself, it's a NaN
        Lines.append('aaaaa')
    else:
        Lines.append(line)
EntropyData['Intimate_embarrassed'] = Lines
len(EntropyData['Intimate_embarrassed'])
# print(EntropyData['nonintimate_Backgroundperceive'].isna().sum())
# print(EntropyData['Nonintimate_Internetforget'].isna().sum())
# print(EntropyData['Nonintimatecreativityimportant'].isna().sum())
# print(EntropyData['Intimate_embarrassed'].isna().sum())
# print(EntropyData['Intimate_forget'].isna().sum())
# print(EntropyData['Intimate_personalcause'].isna().sum())

475

In [10]:
# EntropyData['Intimate_embarrassed'][29:445]
print(EntropyData['nonintimate_Backgroundperceive'].isna().sum())
print(EntropyData['Nonintimate_Internetforget'].isna().sum())
print(EntropyData['Nonintimatecreativityimportant'].isna().sum())
print(EntropyData['Intimate_embarrassed'].isna().sum())
print(EntropyData['Intimate_forget'].isna().sum())
print(EntropyData['Intimate_personalcause'].isna().sum())

0
0
0
0
0
0


In [11]:
print(len(EntropyData['nonintimate_Backgroundperceive']))
print(len(EntropyData['Nonintimate_Internetforget']))
print(len(EntropyData['Nonintimatecreativityimportant']))
print(len(EntropyData['Intimate_embarrassed']))
print(len(EntropyData['Intimate_forget']))
print(len(EntropyData['Intimate_personalcause']))

475
475
475
475
475
475


In [12]:
# Nice! We got rid of the Nans!!! :)

openEndedList = ['nonintimate_Backgroundperceive', 'Nonintimate_Internetforget', 'Nonintimatecreativityimportant',
'Intimate_embarrassed', 'Intimate_forget', 'Intimate_personalcause']

# Now get six columns of dicts
# One for each open-ended category
# dicts will contain all LIWC categories present

for openEndedItem in openEndedList:
    open_List = []
    for line in EntropyData[openEndedItem]:
        line_tokens = tokenize(line)
        line_counts = Counter(category for token in line_tokens for category in parse(token))
        open_List.append(line_counts)
    EntropyData[openEndedItem + 'LIWC'] = open_List

In [13]:
# Now loop through the LIWC dict categories in the EntropyData
# If x = number of LIWC categories, y = number of open ended item columns
# There will be x * y columns, 
# so we can then run analyses on every LIWC category


for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        LIWCline = []
        for line in EntropyData[openEndedItem + 'LIWC']:
            LIWCline.append(line[LIWCcategory])            
        EntropyData[openEndedItem + '_' + LIWCcategory] = LIWCline   

In [14]:
EntropyData.columns.values

array(['Data', 'emotionally', 'treated', 'lovecountry', 'respect',
       'purity', 'math', 'weak', 'unfairly', 'betray', 'traditions',
       'disgusting', 'cruel', 'rights', 'loyalty', 'chaos', 'god',
       'compassion', 'governmentfairly', 'history', 'kidrespect',
       'harmlessdg', 'good', 'killanimal', 'justice', 'family', 'gender',
       'unnatural', 'kill', 'rich_kids', 'team', 'soldier', 'chastity',
       'Conservatism_1', 'CONDITION_H', 'nonintimate_Backgroundperceive',
       'NonintBgpwordcount', 'nonintimate_BackgroundperceiveFirstclick',
       'nonintimate_BackgroundperceiveLastclick',
       'nonintimate_BackgroundperceivePagesubmite',
       'nonintimate_BackgroundperceiveTotalclicks',
       'Nonintimate_Internetforget', 'NonintForgetwordcount',
       'Nonintimate_InternetforgetFirstclick',
       'Nonintimate_InternetforgetLastclick',
       'Nonintimate_InternetforgetPagesubmit',
       'Nonintimate_InternetforgetTotalclicks',
       'Nonintimatecreativityimpor

In [15]:
two_data = pd.DataFrame(EntropyData, index=EntropyData['CONDITION_H'])

In [16]:
from scipy.stats import ttest_ind

In [17]:
# split the dataframe into high versus low entropy conditions
cat1 = EntropyData[EntropyData['CONDITION_H']==1]
cat2 = EntropyData[EntropyData['CONDITION_H']==2]

In [18]:
# results = []
# for LIWCcategory in category_names:
#     for openEndedItem in openEndedList:
#         result = ttest_ind(cat1[openEndedItem + 'LIWC_' + LIWCcategory], cat2[openEndedItem + 'LIWC_' + LIWCcategory])
#         results.append([openEndedItem + 'LIWC_' + LIWCcategory, result])
# print(results)
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        print(openEndedItem + '_' + LIWCcategory)
        print(ttest_ind(cat1[openEndedItem + '_' + LIWCcategory], cat2[openEndedItem + '_' + LIWCcategory]))
        
        

nonintimate_Backgroundperceive_funct
Ttest_indResult(statistic=-0.9747438667312024, pvalue=0.3301855567275659)
Nonintimate_Internetforget_funct
Ttest_indResult(statistic=-1.8305711473287263, pvalue=0.06779359363913698)
Nonintimatecreativityimportant_funct
Ttest_indResult(statistic=-0.64666049307737, pvalue=0.5181653539560883)
Intimate_embarrassed_funct
Ttest_indResult(statistic=-1.6275086752058545, pvalue=0.10429501339974111)
Intimate_forget_funct
Ttest_indResult(statistic=-3.13468657908365, pvalue=0.0018273754553452197)
Intimate_personalcause_funct
Ttest_indResult(statistic=-1.993964496621466, pvalue=0.04672961830870793)
nonintimate_Backgroundperceive_pronoun
Ttest_indResult(statistic=0.5148731277556231, pvalue=0.6068821901519628)
Nonintimate_Internetforget_pronoun
Ttest_indResult(statistic=-1.7537110206707975, pvalue=0.08012769325542209)
Nonintimatecreativityimportant_pronoun
Ttest_indResult(statistic=-0.35325331971128426, pvalue=0.7240559185190727)
Intimate_embarrassed_pronoun
Ttest

Ttest_indResult(statistic=-1.3612243061866929, pvalue=0.17409092706446797)
Intimate_forget_friend
Ttest_indResult(statistic=-1.0255423021775558, pvalue=0.30563155421774013)
Intimate_personalcause_friend
Ttest_indResult(statistic=0.307211327408725, pvalue=0.7588177332528897)
nonintimate_Backgroundperceive_humans
Ttest_indResult(statistic=-1.9784002307286477, pvalue=0.04846331184133898)
Nonintimate_Internetforget_humans
Ttest_indResult(statistic=0.821502094805342, pvalue=0.4117742021992703)
Nonintimatecreativityimportant_humans
Ttest_indResult(statistic=-1.664197569502945, pvalue=0.09673523543990542)
Intimate_embarrassed_humans
Ttest_indResult(statistic=-0.5352872591036486, pvalue=0.5927028057502948)
Intimate_forget_humans
Ttest_indResult(statistic=-1.9524316685511223, pvalue=0.051476485421510135)
Intimate_personalcause_humans
Ttest_indResult(statistic=-0.2966617380251047, pvalue=0.7668549915863175)
nonintimate_Backgroundperceive_affect
Ttest_indResult(statistic=0.5310383788147691, pvalu

Ttest_indResult(statistic=-1.2466249514619123, pvalue=0.21315204008524924)
Intimate_forget_percept
Ttest_indResult(statistic=-1.862486175016065, pvalue=0.06315436248194159)
Intimate_personalcause_percept
Ttest_indResult(statistic=-1.6071381282253712, pvalue=0.10869137012412905)
nonintimate_Backgroundperceive_see
Ttest_indResult(statistic=-5.141227544264964, pvalue=4.002687118735897e-07)
Nonintimate_Internetforget_see
Ttest_indResult(statistic=0.3709511445034187, pvalue=0.710840000890516)
Nonintimatecreativityimportant_see
Ttest_indResult(statistic=0.045066878778977686, pvalue=0.9640730210886145)
Intimate_embarrassed_see
Ttest_indResult(statistic=-1.6962129415423703, pvalue=0.09050353429843754)
Intimate_forget_see
Ttest_indResult(statistic=-0.8477243058269028, pvalue=0.39702027767399806)
Intimate_personalcause_see
Ttest_indResult(statistic=-0.19590741368950343, pvalue=0.8447667688384851)
nonintimate_Backgroundperceive_hear
Ttest_indResult(statistic=3.7191793742178065, pvalue=0.000223844

Ttest_indResult(statistic=1.0621243678240806, pvalue=0.2887212102793946)
Intimate_forget_relig
Ttest_indResult(statistic=-0.7380998016189941, pvalue=0.4608198392120513)
Intimate_personalcause_relig
Ttest_indResult(statistic=1.3697428859072807, pvalue=0.17141706150056923)
nonintimate_Backgroundperceive_death
Ttest_indResult(statistic=nan, pvalue=nan)
Nonintimate_Internetforget_death
Ttest_indResult(statistic=0.24108640218113886, pvalue=0.8095926202295632)
Nonintimatecreativityimportant_death
Ttest_indResult(statistic=-0.4868835580098156, pvalue=0.6265663877668152)
Intimate_embarrassed_death
Ttest_indResult(statistic=-0.6481230087927033, pvalue=0.5172198810012223)
Intimate_forget_death
Ttest_indResult(statistic=-0.06365310059203545, pvalue=0.9492733398044365)
Intimate_personalcause_death
Ttest_indResult(statistic=0.12909202056099547, pvalue=0.897339709134926)
nonintimate_Backgroundperceive_assent
Ttest_indResult(statistic=-1.040924699041621, pvalue=0.2984422592059404)
Nonintimate_Interne

In [19]:
ttest_ind(cat1['nonintimate_Backgroundperceive_see'], cat2['nonintimate_Backgroundperceive_see'])



Ttest_indResult(statistic=-5.141227544264964, pvalue=4.002687118735897e-07)

In [20]:
cat1['nonintimate_Backgroundperceive_see'].describe()

count    247.000000
mean       1.396761
std        1.398486
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        8.000000
Name: nonintimate_Backgroundperceive_see, dtype: float64

In [21]:
cat2['nonintimate_Backgroundperceive_see'].describe()

count    228.000000
mean       2.118421
std        1.657718
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        8.000000
Name: nonintimate_Backgroundperceive_see, dtype: float64

In [22]:
cat2['nonintimate_Backgroundperceive'][1:20]
# note that category 2 is high entropy condition

50                           It has a strong contrast. 
51                                          mess screen
52    Black and white pixels. Snowstorm like an untu...
53    Image is disturbing my eyes; I can't see nothi...
54                It looks grayish white to me...plain.
55    I think I can see some numbers at the bottom. ...
56                                Fuzzy black and white
57                          perhaps slightly disturbing
58    It reminds me of the static from TV when I was...
59                                          blank white
60                              off putting and chaotic
61    Black and white static looking noise, looks a ...
62                                                White
63               Black and white distorted dots/pixels.
64                     it is very busy, hard to look at
65    As distortion, the kind you see when there are...
66    I perceive this background as static on a tele...
67     it is black and white equally shared the 

In [23]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [24]:
# do the 95% confidence intervals

print(mean_confidence_interval(cat1['nonintimate_Backgroundperceive_see']))

(1.3967611336032388, 1.2214944024090246, 1.572027864797453)


In [25]:
print(mean_confidence_interval(cat2['nonintimate_Backgroundperceive_see']))

(2.1184210526315788, 1.9020929361630583, 2.334749169100099)


In [26]:
# Use the code below to turn everything into ratios. Then re-run the t tests.


for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        LIWCRatioLine = []
        for count, line in zip(EntropyData[openEndedItem + '_' + LIWCcategory], EntropyData[openEndedItem]):
#             print(count/len(line.split()))
            LIWCRatioLine.append(count/len(line.split()))      
        EntropyData[LIWCcategory + 'RATIO_' + openEndedItem] = LIWCRatioLine
                  
                  

In [27]:
# split the dataframe into high versus low entropy conditions
cat1 = EntropyData[EntropyData['CONDITION_H']==1]
cat2 = EntropyData[EntropyData['CONDITION_H']==2]

for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        print(openEndedItem + '_' + LIWCcategory)
#         print(ttest_ind(cat1[LIWCcategory + 'RATIO_' + openEndedItem], cat2[LIWCcategory + 'RATIO_' + openEndedItem]))
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
#         print(openEndedItem + '_' + LIWCcategory)
        print(ttest_ind(cat1[LIWCcategory + 'RATIO_' + openEndedItem], cat2[LIWCcategory + 'RATIO_' + openEndedItem]))
          

nonintimate_Backgroundperceive_funct
Nonintimate_Internetforget_funct
Nonintimatecreativityimportant_funct
Intimate_embarrassed_funct
Intimate_forget_funct
Intimate_personalcause_funct
nonintimate_Backgroundperceive_pronoun
Nonintimate_Internetforget_pronoun
Nonintimatecreativityimportant_pronoun
Intimate_embarrassed_pronoun
Intimate_forget_pronoun
Intimate_personalcause_pronoun
nonintimate_Backgroundperceive_ppron
Nonintimate_Internetforget_ppron
Nonintimatecreativityimportant_ppron
Intimate_embarrassed_ppron
Intimate_forget_ppron
Intimate_personalcause_ppron
nonintimate_Backgroundperceive_i
Nonintimate_Internetforget_i
Nonintimatecreativityimportant_i
Intimate_embarrassed_i
Intimate_forget_i
Intimate_personalcause_i
nonintimate_Backgroundperceive_we
Nonintimate_Internetforget_we
Nonintimatecreativityimportant_we
Intimate_embarrassed_we
Intimate_forget_we
Intimate_personalcause_we
nonintimate_Backgroundperceive_you
Nonintimate_Internetforget_you
Nonintimatecreativityimportant_you
Inti

Ttest_indResult(statistic=2.06408996477943, pvalue=0.03955394721207329)
Ttest_indResult(statistic=1.537495668764098, pvalue=0.12484062509104801)
Ttest_indResult(statistic=0.7546013441546939, pvalue=0.4508637457963697)
Ttest_indResult(statistic=-0.4167984263291681, pvalue=0.6770149372732654)
Ttest_indResult(statistic=-0.8973894703714573, pvalue=0.3699676586186349)
Ttest_indResult(statistic=-0.32828038143220634, pvalue=0.7428449496909357)
Ttest_indResult(statistic=-0.9810269866117826, pvalue=0.3270809421412938)
Ttest_indResult(statistic=-0.05605257589275078, pvalue=0.9553235924227377)
Ttest_indResult(statistic=-0.36497072064180985, pvalue=0.7152963917858421)
Ttest_indResult(statistic=0.1434369825504875, pvalue=0.8860061628864336)
Ttest_indResult(statistic=0.044468185045294865, pvalue=0.9645499779648381)
Ttest_indResult(statistic=-0.036522327315809386, pvalue=0.9708812841437906)
Ttest_indResult(statistic=0.3930705147078159, pvalue=0.6944444950272508)
Ttest_indResult(statistic=-0.584472351

Ttest_indResult(statistic=1.1345282614423713, pvalue=0.25714764859539696)
Ttest_indResult(statistic=0.02155905594828002, pvalue=0.982808786030262)
Ttest_indResult(statistic=1.4858118133380083, pvalue=0.1379950912893661)
Ttest_indResult(statistic=1.5478065402255639, pvalue=0.12233766815999927)
Ttest_indResult(statistic=-1.342453133738248, pvalue=0.1800930497723363)
Ttest_indResult(statistic=-0.9971402078353245, pvalue=0.31920629052859334)
Ttest_indResult(statistic=0.4330972790906182, pvalue=0.6651415231277431)
Ttest_indResult(statistic=1.0453234734654446, pvalue=0.2964073969313749)
Ttest_indResult(statistic=-0.7685152476244959, pvalue=0.4425646651202365)
Ttest_indResult(statistic=-0.04022709129800142, pvalue=0.967929052942778)
Ttest_indResult(statistic=1.5628691321911254, pvalue=0.1187521544159059)
Ttest_indResult(statistic=0.5867012856820548, pvalue=0.5576842454970619)
Ttest_indResult(statistic=-0.6231824356373675, pvalue=0.5334650038710111)
Ttest_indResult(statistic=-0.426044236238901

In [28]:
EntropyData.columns.values

array(['Data', 'emotionally', 'treated', 'lovecountry', 'respect',
       'purity', 'math', 'weak', 'unfairly', 'betray', 'traditions',
       'disgusting', 'cruel', 'rights', 'loyalty', 'chaos', 'god',
       'compassion', 'governmentfairly', 'history', 'kidrespect',
       'harmlessdg', 'good', 'killanimal', 'justice', 'family', 'gender',
       'unnatural', 'kill', 'rich_kids', 'team', 'soldier', 'chastity',
       'Conservatism_1', 'CONDITION_H', 'nonintimate_Backgroundperceive',
       'NonintBgpwordcount', 'nonintimate_BackgroundperceiveFirstclick',
       'nonintimate_BackgroundperceiveLastclick',
       'nonintimate_BackgroundperceivePagesubmite',
       'nonintimate_BackgroundperceiveTotalclicks',
       'Nonintimate_Internetforget', 'NonintForgetwordcount',
       'Nonintimate_InternetforgetFirstclick',
       'Nonintimate_InternetforgetLastclick',
       'Nonintimate_InternetforgetPagesubmit',
       'Nonintimate_InternetforgetTotalclicks',
       'Nonintimatecreativityimpor

In [29]:
#stats.pearsonr(EntropyData['VIXCLS'], EntropyData['Conservatism_1'])
import scipy.stats  as stats
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        print(stats.pearsonr(EntropyData[LIWCcategory + 'RATIO_' + openEndedItem], EntropyData['Conservatism_1']))
for LIWCcategory in category_names:    
    for openEndedItem in openEndedList:
        print(LIWCcategory + 'RATIO_' + openEndedItem)
for LIWCcategory in category_names:    
    for openEndedItem in openEndedList:
        print(openEndedItem)
# EntropyData.columns.values
          

(0.08751825943164024, 0.05664381104613729)
(0.09984604446351325, 0.0295702071978863)
(0.030949422080149853, 0.5010039106418309)
(-0.016356991862403934, 0.7221574556330186)
(-0.022801620718894623, 0.6201026257587404)
(-0.002215364678683242, 0.9615922360824292)
(0.09162502159490513, 0.04594921330177313)
(-0.029945345579236216, 0.5150012169214409)
(0.12121788405759766, 0.008176807135854353)
(-0.07042489246365928, 0.12534129569706645)
(0.02896078703106626, 0.528920479339336)
(0.04460321521642689, 0.3320334904288951)
(0.08253059871165187, 0.07232924001810358)
(-0.038692465833523745, 0.40013733439823407)
(0.0992394834849802, 0.03057839377408891)
(-0.0667018338509723, 0.1466352846287636)
(0.01596378997720518, 0.7285701674688333)
(0.056923912251940795, 0.21558133123017523)
(0.08576908362610373, 0.06178831551933311)
(0.043854578126162, 0.34022084465631663)
(0.06384808286429852, 0.16474514707151464)
(-0.04017860418593916, 0.3822718330346192)
(0.016253294280715036, 0.7238467878644346)
(0.06935486

/Users/robertlewis/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den



(0.08879398120586221, 0.05312019188639698)
(0.014205999733261842, 0.7574642270764103)
(0.10921176430189414, 0.01726243643075251)
(nan, 1.0)
(0.09915991234884053, 0.03071281053775081)
(0.02678315914252873, 0.560370078770734)
(0.04074450930490394, 0.37559905908194235)
(-0.03826882201808785, 0.4053203501711167)
(0.03267510667297283, 0.4774246036947518)
(nan, 1.0)
(nan, 1.0)
(nan, 1.0)
(nan, 1.0)
(nan, 1.0)
(nan, 1.0)
functRATIO_nonintimate_Backgroundperceive
functRATIO_Nonintimate_Internetforget
functRATIO_Nonintimatecreativityimportant
functRATIO_Intimate_embarrassed
functRATIO_Intimate_forget
functRATIO_Intimate_personalcause
pronounRATIO_nonintimate_Backgroundperceive
pronounRATIO_Nonintimate_Internetforget
pronounRATIO_Nonintimatecreativityimportant
pronounRATIO_Intimate_embarrassed
pronounRATIO_Intimate_forget
pronounRATIO_Intimate_personalcause
ppronRATIO_nonintimate_Backgroundperceive
ppronRATIO_Nonintimate_Internetforget
ppronRATIO_Nonintimatecreativityimportant
ppronRATIO_Intima

In [30]:
# compute progressivism
EntropyData['care_fair_avg'] = EntropyData[['emotionally', 'treated', 'weak', 'unfairly', 
               'cruel', 'rights', 'compassion', 'governmentfairly', 
               'killanimal', 'justice',
               'kill', 'rich_kids']].mean(axis=1)
EntropyData['loy_auth__pur_avg'] = EntropyData[['lovecountry', 'respect',
       'purity', 'betray', 'traditions',
       'disgusting', 'loyalty', 'chaos', 'god',
       'history', 'kidrespect',
       'harmlessdg', 'family', 
       'unnatural', 'team', 'soldier', 'chastity']].mean(axis=1)
EntropyData['progressivism'] =  EntropyData['care_fair_avg'] - EntropyData['loy_auth__pur_avg'] 

# ALL ITEMS
# 'emotionally', 'treated', 'lovecountry', 'respect',
#        'purity', 'math', 'weak', 'unfairly', 'betray', 'traditions',
#        'disgusting', 'cruel', 'rights', 'loyalty', 'chaos', 'god',
#        'compassion', 'governmentfairly', 'history', 'kidrespect',
#        'harmlessdg', 'good', 'killanimal', 'justice', 'family', 'gender',
#        'unnatural', 'kill', 'rich_kids', 'team', 'soldier', 'chastity'
    
    

In [31]:
EntropyData.head()

,Data,emotionally,treated,lovecountry,respect,purity,math,weak,unfairly,betray,...,nonflRATIO_Intimate_personalcause,fillerRATIO_nonintimate_Backgroundperceive,fillerRATIO_Nonintimate_Internetforget,fillerRATIO_Nonintimatecreativityimportant,fillerRATIO_Intimate_embarrassed,fillerRATIO_Intimate_forget,fillerRATIO_Intimate_personalcause,care_fair_avg,loy_auth__pur_avg,progressivism
0,0,6,6,4,2,5,1,6,5,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.250000,3.529412,1.720588
1,0,5,2,1,4,6,1,5,6,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.583333,3.470588,1.112745
2,0,4,5,1,2,1,2,4,5,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.166667,1.764706,3.401961
3,0,6,6,1,4,5,1,6,6,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.583333,3.176471,1.406863
4,0,4,4,1,2,4,1,5,5,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,2.823529,1.176471


In [32]:
EntropyData.columns.values

array(['Data', 'emotionally', 'treated', 'lovecountry', 'respect',
       'purity', 'math', 'weak', 'unfairly', 'betray', 'traditions',
       'disgusting', 'cruel', 'rights', 'loyalty', 'chaos', 'god',
       'compassion', 'governmentfairly', 'history', 'kidrespect',
       'harmlessdg', 'good', 'killanimal', 'justice', 'family', 'gender',
       'unnatural', 'kill', 'rich_kids', 'team', 'soldier', 'chastity',
       'Conservatism_1', 'CONDITION_H', 'nonintimate_Backgroundperceive',
       'NonintBgpwordcount', 'nonintimate_BackgroundperceiveFirstclick',
       'nonintimate_BackgroundperceiveLastclick',
       'nonintimate_BackgroundperceivePagesubmite',
       'nonintimate_BackgroundperceiveTotalclicks',
       'Nonintimate_Internetforget', 'NonintForgetwordcount',
       'Nonintimate_InternetforgetFirstclick',
       'Nonintimate_InternetforgetLastclick',
       'Nonintimate_InternetforgetPagesubmit',
       'Nonintimate_InternetforgetTotalclicks',
       'Nonintimatecreativityimpor

In [33]:
import scipy.stats  as stats
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        print(stats.pearsonr(EntropyData[LIWCcategory + 'RATIO_' + openEndedItem], EntropyData['progressivism']))
for LIWCcategory in category_names:    
    for openEndedItem in openEndedList:
        print(LIWCcategory + 'RATIO_' + openEndedItem)
for LIWCcategory in category_names:    
    for openEndedItem in openEndedList:
        print(openEndedItem)

(-0.03161116484376743, 0.49188996794726936)
(-0.11953772592222271, 0.00911377423712056)
(0.04644366476335953, 0.3124500725734036)
(0.04101806035081018, 0.3723993980517114)
(0.07887095411222426, 0.08596104158955864)
(0.093291330626577, 0.042121847512756966)
(-0.06525760154279667, 0.15560095848050423)
(0.04256586853050347, 0.35461391165262013)
(-0.06324413689726849, 0.16878441726384713)
(-0.020633050626509025, 0.6537564218070402)
(0.03492096902288411, 0.447666417165575)
(-0.07020816246223577, 0.12651109056798504)
(-0.05378699195265272, 0.24199341682378486)
(0.04738759158842374, 0.3027066053565595)
(-0.03386912731628811, 0.46147091946261276)
(-0.032753817308605944, 0.4763637486831308)
(0.020783596199384583, 0.6513962703537959)
(-0.08018716074323695, 0.08083767369636499)
(-0.055850021385607886, 0.22437968518249263)
(0.001425460791935024, 0.9752812163100754)
(-0.022250563464175872, 0.6285831970650038)
(-0.02822006317441299, 0.5395168805145951)
(0.03751275073861455, 0.41466918267071107)
(-0.

/Users/robertlewis/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarrassed
Intimate_forget
Intimate_personalcause
nonintimate_Backgroundperceive
Nonintimate_Internetforget
Nonintimatecreativityimportant
Intimate_embarras

In [34]:
# ancovas ... will play around with ancova versus regression with dummy 
# (same analsyiss but output looks different
from statsmodels.formula.api import ols

formula = 'Intimate_forget_pronoun ~ Conservatism_1 * C(CONDITION_H)'  # ANCOVA formula
lm = ols(formula, data=EntropyData)
results = lm.fit()
print(results.summary())



                               OLS Regression Results                              
Dep. Variable:     Intimate_forget_pronoun   R-squared:                       0.021
Model:                                 OLS   Adj. R-squared:                  0.015
Method:                      Least Squares   F-statistic:                     3.374
Date:                     Wed, 16 Jan 2019   Prob (F-statistic):             0.0184
Time:                             00:22:33   Log-Likelihood:                -1540.0
No. Observations:                      475   AIC:                             3088.
Df Residuals:                          471   BIC:                             3105.
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [35]:
# outputList = []
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        formula = LIWCcategory + 'RATIO_' + openEndedItem + ' ~ Conservatism_1 * C(CONDITION_H)'  # ANCOVA formula
        lm = ols(formula, data=EntropyData)
        results = lm.fit()
        print(results.summary())




                                        OLS Regression Results                                       
Dep. Variable:     functRATIO_nonintimate_Backgroundperceive   R-squared:                       0.053
Model:                                                   OLS   Adj. R-squared:                  0.047
Method:                                        Least Squares   F-statistic:                     8.835
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):           1.04e-05
Time:                                               00:22:33   Log-Likelihood:                 46.604
No. Observations:                                        475   AIC:                            -85.21
Df Residuals:                                            471   BIC:                            -68.55
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                         OLS Regression Results                                        
Dep. Variable:     pronounRATIO_nonintimate_Backgroundperceive   R-squared:                       0.008
Model:                                                     OLS   Adj. R-squared:                  0.002
Method:                                          Least Squares   F-statistic:                     1.333
Date:                                         Wed, 16 Jan 2019   Prob (F-statistic):              0.263
Time:                                                 00:22:33   Log-Likelihood:                 392.34
No. Observations:                                          475   AIC:                            -776.7
Df Residuals:                                              471   BIC:                            -760.0
Df Model:                                                    3                                         
Covariance Type:                                     nonrobust  

                                        OLS Regression Results                                       
Dep. Variable:     ppronRATIO_nonintimate_Backgroundperceive   R-squared:                       0.008
Model:                                                   OLS   Adj. R-squared:                  0.002
Method:                                        Least Squares   F-statistic:                     1.263
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):              0.286
Time:                                               00:22:34   Log-Likelihood:                 703.72
No. Observations:                                        475   AIC:                            -1399.
Df Residuals:                                            471   BIC:                            -1383.
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                    OLS Regression Results                                   
Dep. Variable:     ppronRATIO_Intimate_personalcause   R-squared:                       0.005
Model:                                           OLS   Adj. R-squared:                 -0.001
Method:                                Least Squares   F-statistic:                    0.8610
Date:                               Wed, 16 Jan 2019   Prob (F-statistic):              0.461
Time:                                       00:22:34   Log-Likelihood:                 599.35
No. Observations:                                475   AIC:                            -1191.
Df Residuals:                                    471   BIC:                            -1174.
Df Model:                                          3                                         
Covariance Type:                           nonrobust                                         
                                         coef    std err    

                              OLS Regression Results                              
Dep. Variable:     iRATIO_Intimate_forget   R-squared:                       0.001
Model:                                OLS   Adj. R-squared:                 -0.005
Method:                     Least Squares   F-statistic:                    0.1437
Date:                    Wed, 16 Jan 2019   Prob (F-statistic):              0.934
Time:                            00:22:34   Log-Likelihood:                 598.50
No. Observations:                     475   AIC:                            -1189.
Df Residuals:                         471   BIC:                            -1172.
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

...

                                        OLS Regression Results                                       
Dep. Variable:     religRATIO_Nonintimatecreativityimportant   R-squared:                       0.004
Model:                                                   OLS   Adj. R-squared:                 -0.002
Method:                                        Least Squares   F-statistic:                    0.6376
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):              0.591
Time:                                               00:22:48   Log-Likelihood:                 1771.5
No. Observations:                                        475   AIC:                            -3535.
Df Residuals:                                            471   BIC:                            -3518.
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                OLS Regression Results                                
Dep. Variable:     deathRATIO_Intimate_forget   R-squared:                       0.012
Model:                                    OLS   Adj. R-squared:                  0.005
Method:                         Least Squares   F-statistic:                     1.868
Date:                        Wed, 16 Jan 2019   Prob (F-statistic):              0.134
Time:                                00:22:48   Log-Likelihood:                 679.63
No. Observations:                         475   AIC:                            -1351.
Df Residuals:                             471   BIC:                            -1335.
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

                                    OLS Regression Results                                    
Dep. Variable:     assentRATIO_Intimate_personalcause   R-squared:                       0.014
Model:                                            OLS   Adj. R-squared:                  0.008
Method:                                 Least Squares   F-statistic:                     2.261
Date:                                Wed, 16 Jan 2019   Prob (F-statistic):             0.0806
Time:                                        00:22:49   Log-Likelihood:                 1429.1
No. Observations:                                 475   AIC:                            -2850.
Df Residuals:                                     471   BIC:                            -2834.
Df Model:                                           3                                         
Covariance Type:                            nonrobust                                         
                                         coef    s

                                        OLS Regression Results                                        
Dep. Variable:     fillerRATIO_nonintimate_Backgroundperceive   R-squared:                         nan
Model:                                                    OLS   Adj. R-squared:                    nan
Method:                                         Least Squares   F-statistic:                       nan
Date:                                        Wed, 16 Jan 2019   Prob (F-statistic):                nan
Time:                                                00:22:49   Log-Likelihood:                    inf
No. Observations:                                         475   AIC:                              -inf
Df Residuals:                                             471   BIC:                              -inf
Df Model:                                                   3                                         
Covariance Type:                                    nonrobust            

                                 OLS Regression Results                                
Dep. Variable:     fillerRATIO_Intimate_forget   R-squared:                         nan
Model:                                     OLS   Adj. R-squared:                    nan
Method:                          Least Squares   F-statistic:                       nan
Date:                         Wed, 16 Jan 2019   Prob (F-statistic):                nan
Time:                                 00:22:49   Log-Likelihood:                    inf
No. Observations:                          475   AIC:                              -inf
Df Residuals:                              471   BIC:                              -inf
Df Model:                                    3                                         
Covariance Type:                     nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [36]:
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        formula = LIWCcategory + 'RATIO_' + openEndedItem + ' ~ progressivism * C(CONDITION_H)'  # ANCOVA formula
        lm = ols(formula, data=EntropyData)
        results = lm.fit()
        print(results.summary())

                                        OLS Regression Results                                       
Dep. Variable:     functRATIO_nonintimate_Backgroundperceive   R-squared:                       0.056
Model:                                                   OLS   Adj. R-squared:                  0.050
Method:                                        Least Squares   F-statistic:                     9.390
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):           4.88e-06
Time:                                               00:22:50   Log-Likelihood:                 47.397
No. Observations:                                        475   AIC:                            -86.79
Df Residuals:                                            471   BIC:                            -70.14
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                OLS Regression Results                                
Dep. Variable:     functRATIO_Intimate_forget   R-squared:                       0.007
Model:                                    OLS   Adj. R-squared:                  0.001
Method:                         Least Squares   F-statistic:                     1.103
Date:                        Wed, 16 Jan 2019   Prob (F-statistic):              0.348
Time:                                00:22:50   Log-Likelihood:                 313.40
No. Observations:                         475   AIC:                            -618.8
Df Residuals:                             471   BIC:                            -602.1
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

                                         OLS Regression Results                                        
Dep. Variable:     pronounRATIO_Nonintimatecreativityimportant   R-squared:                       0.004
Model:                                                     OLS   Adj. R-squared:                 -0.002
Method:                                          Least Squares   F-statistic:                    0.6694
Date:                                         Wed, 16 Jan 2019   Prob (F-statistic):              0.571
Time:                                                 00:22:51   Log-Likelihood:                 516.82
No. Observations:                                          475   AIC:                            -1026.
Df Residuals:                                              471   BIC:                            -1009.
Df Model:                                                    3                                         
Covariance Type:                                     nonrobust  

                                        OLS Regression Results                                       
Dep. Variable:     ppronRATIO_nonintimate_Backgroundperceive   R-squared:                       0.003
Model:                                                   OLS   Adj. R-squared:                 -0.003
Method:                                        Least Squares   F-statistic:                    0.4635
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):              0.708
Time:                                               00:22:51   Log-Likelihood:                 702.52
No. Observations:                                        475   AIC:                            -1397.
Df Residuals:                                            471   BIC:                            -1380.
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                OLS Regression Results                                
Dep. Variable:     ppronRATIO_Intimate_forget   R-squared:                       0.000
Model:                                    OLS   Adj. R-squared:                 -0.006
Method:                         Least Squares   F-statistic:                   0.07793
Date:                        Wed, 16 Jan 2019   Prob (F-statistic):              0.972
Time:                                00:22:52   Log-Likelihood:                 590.10
No. Observations:                         475   AIC:                            -1172.
Df Residuals:                             471   BIC:                            -1156.
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

...

                                       OLS Regression Results                                       
Dep. Variable:     homeRATIO_Nonintimatecreativityimportant   R-squared:                       0.002
Model:                                                  OLS   Adj. R-squared:                 -0.004
Method:                                       Least Squares   F-statistic:                    0.3115
Date:                                      Wed, 16 Jan 2019   Prob (F-statistic):              0.817
Time:                                              00:23:10   Log-Likelihood:                 2292.4
No. Observations:                                       475   AIC:                            -4577.
Df Residuals:                                           471   BIC:                            -4560.
Df Model:                                                 3                                         
Covariance Type:                                  nonrobust                                

                                    OLS Regression Results                                   
Dep. Variable:     moneyRATIO_Intimate_personalcause   R-squared:                       0.010
Model:                                           OLS   Adj. R-squared:                  0.004
Method:                                Least Squares   F-statistic:                     1.655
Date:                               Wed, 16 Jan 2019   Prob (F-statistic):              0.176
Time:                                       00:23:10   Log-Likelihood:                 980.34
No. Observations:                                475   AIC:                            -1953.
Df Residuals:                                    471   BIC:                            -1936.
Df Model:                                          3                                         
Covariance Type:                           nonrobust                                         
                                        coef    std err     

                                   OLS Regression Results                                  
Dep. Variable:     deathRATIO_Intimate_embarrassed   R-squared:                       0.003
Model:                                         OLS   Adj. R-squared:                 -0.004
Method:                              Least Squares   F-statistic:                    0.4008
Date:                             Wed, 16 Jan 2019   Prob (F-statistic):              0.753
Time:                                     00:23:10   Log-Likelihood:                 2252.0
No. Observations:                              475   AIC:                            -4496.
Df Residuals:                                  471   BIC:                            -4479.
Df Model:                                        3                                         
Covariance Type:                         nonrobust                                         
                                        coef    std err          t      P>|t|   

                                        OLS Regression Results                                       
Dep. Variable:     nonflRATIO_nonintimate_Backgroundperceive   R-squared:                         nan
Model:                                                   OLS   Adj. R-squared:                    nan
Method:                                        Least Squares   F-statistic:                       nan
Date:                                       Wed, 16 Jan 2019   Prob (F-statistic):                nan
Time:                                               00:23:11   Log-Likelihood:                    inf
No. Observations:                                        475   AIC:                              -inf
Df Residuals:                                            471   BIC:                              -inf
Df Model:                                                  3                                         
Covariance Type:                                   nonrobust                      

                                        OLS Regression Results                                        
Dep. Variable:     fillerRATIO_Nonintimatecreativityimportant   R-squared:                         nan
Model:                                                    OLS   Adj. R-squared:                    nan
Method:                                         Least Squares   F-statistic:                       nan
Date:                                        Wed, 16 Jan 2019   Prob (F-statistic):                nan
Time:                                                00:23:11   Log-Likelihood:                    inf
No. Observations:                                         475   AIC:                              -inf
Df Residuals:                                             471   BIC:                              -inf
Df Model:                                                   3                                         
Covariance Type:                                    nonrobust            

In [37]:
cols = list(EntropyData.columns)
#  so we can get standardized regression coefficients we need to make zscore columns
# EntropyData.dropna()
for col in cols:
    col_zscore = col + '_zscore'
    if np.issubdtype(EntropyData[col].dtype, np.number):
        EntropyData[col_zscore] = (EntropyData[col] - EntropyData[col].mean())/EntropyData[col].std(ddof=0)
# EntropyData = EntropyData.dropna()
EntropyData = EntropyData.dropna(axis=1, how='all') # Drop the columns where all elements are missing.
    
    

In [39]:
resultsList = []
resultsListItem = []
resultsListLiwc = []
resultsListRsquared = []
resultsListCondition = []
resultsListProgressivism = []
resultsListInteraction = []
resultsListConditionP = []
resultsListProgressivismP = []
resultsListInteractionP = []
tagList = []
for LIWCcategory in category_names:
    for openEndedItem in openEndedList:
        if LIWCcategory + 'RATIO_' + openEndedItem + '_zscore' in EntropyData.columns:
            formula = LIWCcategory + 'RATIO_' + openEndedItem + '_zscore ~ progressivism_zscore * C(CONDITION_H)'  # ANCOVA formula
            lm = ols(formula, data=EntropyData)
            results = lm.fit()
            tagList.append(LIWCcategory + 'RATIO_' + openEndedItem + '_zscore')
            resultsListRsquared.append(results.rsquared)
            resultsListItem.append(openEndedItem)
            resultsListLiwc.append(LIWCcategory)
            resultsListCondition.append(results.params[1])
            resultsListProgressivism.append(results.params.progressivism_zscore)
            resultsListInteraction.append(results.params[3])
            resultsListConditionP.append(results.pvalues[1])
            resultsListProgressivismP.append(results.pvalues[2])
            resultsListInteractionP.append(results.pvalues[3])
#         resultsList.item =
#         pd.DataFrame({
#             "item": openEndedItem,
#             "liwc": LIWCcategory})
print(len(resultsListItem))
print(len(resultsListLiwc))
print(len(resultsListRsquared))
# need to transpose ...
myResults = pd.DataFrame(np.array([tagList, resultsListItem, resultsListLiwc, resultsListCondition, resultsListProgressivism, resultsListInteraction, resultsListConditionP, resultsListProgressivismP, resultsListInteractionP]).T.tolist(), 
                         columns=["tag", "item", "liwc", "condition", "progressivism", "interaction", "conditionP", "progressivismP", "interactionP"])
myResults['rsquared'] = resultsListRsquared
# myResults.rename(index=str, columns={"A": "a", "B": "c"})
myResults.head()
  
    
    
    
    
    
    
    
    

369
369
369


,tag,item,liwc,condition,progressivism,interaction,conditionP,progressivismP,interactionP,rsquared
0,functRATIO_nonintimate_Backgroundperceive_zscore,nonintimate_Backgroundperceive,funct,0.4383089941020045,0.04162127335948601,-0.1764893572168097,1.3988136069439916e-06,0.49795466292562984,0.049937406273498144,0.056434
1,functRATIO_Nonintimate_Internetforget_zscore,Nonintimate_Internetforget,funct,0.02520272428926478,-0.12792302150421492,0.016813766191186096,0.7834204531325947,0.04193282812458161,0.854697542654812,0.014519
2,functRATIO_Nonintimatecreativityimportant_zscore,Nonintimatecreativityimportant,funct,0.04414908189902156,-0.005142457564288262,0.10843139415906876,0.631734849820061,0.9349775694368669,0.24006572853067182,0.005575
3,functRATIO_Intimate_embarrassed_zscore,Intimate_embarrassed,funct,-0.01463530378003534,-0.032138252192224555,0.15724904733521305,0.8736030528950112,0.6097795990785919,0.08831769127151311,0.007875
4,functRATIO_Intimate_forget_zscore,Intimate_forget,funct,0.05184049383344996,0.06924748505339855,0.018264601717211917,0.5733288802491958,0.2719123394686395,0.8429102730213782,0.006975


In [40]:
columnvals = EntropyData.columns.values
for item in columnvals:
    print(item)


Data
emotionally
treated
lovecountry
respect
purity
math
weak
unfairly
betray
traditions
disgusting
cruel
rights
loyalty
chaos
god
compassion
governmentfairly
history
kidrespect
harmlessdg
good
killanimal
justice
family
gender
unnatural
kill
rich_kids
team
soldier
chastity
Conservatism_1
CONDITION_H
nonintimate_Backgroundperceive
NonintBgpwordcount
nonintimate_BackgroundperceiveFirstclick
nonintimate_BackgroundperceiveLastclick
nonintimate_BackgroundperceivePagesubmite
nonintimate_BackgroundperceiveTotalclicks
Nonintimate_Internetforget
NonintForgetwordcount
Nonintimate_InternetforgetFirstclick
Nonintimate_InternetforgetLastclick
Nonintimate_InternetforgetPagesubmit
Nonintimate_InternetforgetTotalclicks
Nonintimatecreativityimportant
NonintcreatWordcount
NonintimatecreativityimportantFirstclick
NonintimatecreativityimportantLastclick
NonintimatecreativityimportantPagesubmite
NonintimatecreativityimportantTotalclicks
Intimate_embarrassed
Intembawordcount
Intimate_embarrassedFirstclick
I

spaceRATIO_nonintimate_Backgroundperceive_zscore
spaceRATIO_Nonintimate_Internetforget_zscore
spaceRATIO_Nonintimatecreativityimportant_zscore
spaceRATIO_Intimate_embarrassed_zscore
spaceRATIO_Intimate_forget_zscore
spaceRATIO_Intimate_personalcause_zscore
timeRATIO_nonintimate_Backgroundperceive_zscore
timeRATIO_Nonintimate_Internetforget_zscore
timeRATIO_Nonintimatecreativityimportant_zscore
timeRATIO_Intimate_embarrassed_zscore
timeRATIO_Intimate_forget_zscore
timeRATIO_Intimate_personalcause_zscore
workRATIO_nonintimate_Backgroundperceive_zscore
workRATIO_Nonintimate_Internetforget_zscore
workRATIO_Nonintimatecreativityimportant_zscore
workRATIO_Intimate_embarrassed_zscore
workRATIO_Intimate_forget_zscore
workRATIO_Intimate_personalcause_zscore
achieveRATIO_nonintimate_Backgroundperceive_zscore
achieveRATIO_Nonintimate_Internetforget_zscore
achieveRATIO_Nonintimatecreativityimportant_zscore
achieveRATIO_Intimate_embarrassed_zscore
achieveRATIO_Intimate_forget_zscore
achieveRATIO_In

In [42]:
sortedByPint = myResults.sort_values(by='interactionP', ascending=True)
sortedByPint[1:20]
# sortedByPprog = myResults.sort_values(by='progressivismP', ascending=True)
# sortedByPprog

,tag,item,liwc,condition,progressivism,interaction,conditionP,progressivismP,interactionP,rsquared
27,weRATIO_Intimate_embarrassed_zscore,Intimate_embarrassed,we,0.22089372471263394,-0.0032034904839626306,0.30894438795660123,0.014075198164957836,0.9583710799705626,0.0006283686033579113,0.057277
178,angerRATIO_Intimate_personalcause_zscore,Intimate_personalcause,anger,0.030748919106178753,-0.1457416499469571,0.2679627582143205,0.7368532593824404,0.020304305792820002,0.0035989813543875677,0.018492
265,bioRATIO_Nonintimatecreativityimportant_zscore,Nonintimatecreativityimportant,bio,0.011015436186945915,0.30716566334867706,-0.22772791182378552,0.9024423043845876,8.217049680080602e-07,0.011664956427698986,0.053315
23,iRATIO_Intimate_personalcause_zscore,Intimate_personalcause,i,-0.06839804195813032,-0.25909386103182386,0.2241077772866767,0.45038805648150615,3.458806966604209e-05,0.013799219798328401,0.037895
318,workRATIO_Nonintimate_Internetforget_zscore,Nonintimate_Internetforget,work,-0.06560097874688903,0.20861714210389748,-0.21449206565690532,0.4722961477932939,0.0008963209452762572,0.019249008845845045,0.023991
177,angerRATIO_Intimate_forget_zscore,Intimate_forget,anger,0.08808989744283763,0.04244021215157558,-0.208708703585933,0.3366579497821952,0.49869518497249143,0.02331840857980262,0.015548
128,swearRATIO_Intimate_personalcause_zscore,Intimate_personalcause,swear,-0.054999592291820484,-0.13288526146772434,0.20109758895327268,0.5491097996704034,0.03481916232868884,0.02907976862690971,0.012385
35,youRATIO_Intimate_personalcause_zscore,Intimate_personalcause,you,-0.017510591274939317,0.07690737189892644,-0.1966707546156505,0.8488908167716511,0.22177042938711244,0.03301006560204868,0.009931
28,weRATIO_Intimate_forget_zscore,Intimate_forget,we,0.05898444649606043,-0.013077718524290367,0.18408711377901724,0.5200472440991255,0.834884327018611,0.0453854183924958,0.014812
140,friendRATIO_Intimate_embarrassed_zscore,Intimate_embarrassed,friend,0.08886727745509959,-0.06533730530082305,0.1844213125860207,0.3335357274500811,0.298900947576693,0.045413822196235784,0.010970


In [44]:
# E explore those interactions

cat1 = EntropyData[EntropyData['CONDITION_H']==1]
cat2 = EntropyData[EntropyData['CONDITION_H']==2]

for var in ['sexualRATIO_Nonintimatecreativityimportant_zscore', 'angerRATIO_Intimate_personalcause_zscore']:
    print("low entropy")
    print(stats.pearsonr(cat1[var], cat1['progressivism']))
    print("high entropy")    
    print(stats.pearsonr(cat2[var], cat2['progressivism']))

    
#     for openEndedItem in openEndedList:
#         if LIWCcategory + 'RATIO_' + openEndedItem + '_zscore' in EntropyData.columns:
#         print(LIWCcategory + 'RATIO_' + openEndedItem)
        

low entropy
(0.25068397404005055, 6.787418632325406e-05)
high entropy
(-0.04077005932309902, 0.5402165873888793)
low entropy
(-0.1515671292086292, 0.017136347678793178)
high entropy
(0.11710254686146028, 0.07763656777306195)
